# Clonamos el repositorio con los modelos y herramientas

In [1]:
!git clone https://github.com/dannasalazar11/Msc_thesis.git

Cloning into 'Msc_thesis'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 23 (delta 6), reused 16 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 8.69 KiB | 2.17 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [2]:
import sys
sys.path.append('/kaggle/working/Msc_thesis')

from gmrrnet_adhd.utils import get_segmented_data, train_L24O_cv
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

2025-04-28 16:24:08.147872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745857448.427926      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745857448.504835      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Importar base de datos segmentada (Segmentos de 4 seg con translape del 50%, es decir, de 2 seg)

In [3]:
X, y, sbjs = get_segmented_data()
X.shape, y.shape, len(sbjs)

((8213, 19, 512), (8213, 2), 8213)

# Importamos el modelo y definimos los hiperparámetros

In [4]:
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredError
from gmrrnet_adhd.models.EEGNet import EEGNet

model_name = 'EEGNet'
model_args = {
    'Chans' : 19,
    'Samples' : 512,
    'nb_classes': 2,
    'dropoutRate': 0.5,
    'kernLength': 32,
    'F1': 8,
    'D': 2,
    'F2': 16,
    'norm_rate': 0.25,
    'dropoutType': 'Dropout'
}

compile_args = {
    'loss': CategoricalCrossentropy(),  # Alternativa: 'mse' o MeanSquaredError()
    'init_lr': 1e-2,
    'metrics' : ['categorical_accuracy']
}

model = EEGNet(**model_args)

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-04-28 16:24:29.107843: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 19, 512, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cast (Cast)                          │ (None, 19, 512, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2D_1 (Conv2D)                    │ (None, 19, 512, 8)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 19, 512, 8)          │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Depth_wise_Conv2D_1                  │ (None, 1, 512, 16)          │             304 │
│ (DepthwiseConv2D)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 1, 512, 16)          │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 1, 512, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d (AveragePooling2D) │ (None, 1, 128, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1, 128, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Separable_Conv2D_1 (SeparableConv2D) │ (None, 1, 128, 16)          │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 1, 128, 16)          │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 1, 128, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_1                  │ (None, 1, 16, 16)           │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1, 16, 16)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 2)                   │             514 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ out_activation (Activation)          │ (None, 2)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,746 (6.82 KB)

 Trainable params: 1,666 (6.51 KB)

 Non-trainable params: 80 (320.00 B)

# Resultados - Leave 24 Subjects Out +  CV (cada fold se corre 5 veces)

In [5]:
import os

import pickle

with open("/kaggle/input/ieee-tdah-control-database/folds.pkl", "rb") as f:
    folds = pickle.load(f)

In [6]:
results = train_L24O_cv(EEGNet, X, y, sbjs, model_args, compile_args, folds)

Fold 1/5
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step
Fold metrics: {'accuracy': 0.8023335337638855, 'recall': 0.8093640022958297, 'precision': 0.836584008117957, 'kappa': 0.6096747847991055, 'auc': 0.9009833741145197}
Fold 2/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


53/53 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step
Fold metrics: {'accuracy': 0.7793765068054199, 'recall': 0.7601221021151445, 'precision': 0.8005795432350702, 'kappa': 0.5383442111770784, 'auc': 0.8982758067877583}
Fold 3/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step
Fold metrics: {'accuracy': 0.8823529481887817, 'recall': 0.8529599754475157, 'precision': 0.9167095958044448, 'kappa': 0.7420784711438787, 'auc': 0.8897913706024032}
Fold 4/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


53/53 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step
Fold metrics: {'accuracy': 0.895080029964447, 'recall': 0.8978158723602003, 'precision': 0.8919834479483766, 'kappa': 0.7879339255010664, 'auc': 0.9758133107290179}
Fold 5/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step
Fold metrics: {'accuracy': 0.9142681956291199, 'recall': 0.9121755902793414, 'precision': 0.9257849084221821, 'kappa': 0.8277169137650131, 'auc': 0.9884083096921392}


In [7]:
results

[{'accuracy': 0.8023335337638855,
  'recall': 0.8093640022958297,
  'precision': 0.836584008117957,
  'kappa': 0.6096747847991055,
  'auc': 0.9009833741145197},
 {'accuracy': 0.7793765068054199,
  'recall': 0.7601221021151445,
  'precision': 0.8005795432350702,
  'kappa': 0.5383442111770784,
  'auc': 0.8982758067877583},
 {'accuracy': 0.8823529481887817,
  'recall': 0.8529599754475157,
  'precision': 0.9167095958044448,
  'kappa': 0.7420784711438787,
  'auc': 0.8897913706024032},
 {'accuracy': 0.895080029964447,
  'recall': 0.8978158723602003,
  'precision': 0.8919834479483766,
  'kappa': 0.7879339255010664,
  'auc': 0.9758133107290179},
 {'accuracy': 0.9142681956291199,
  'recall': 0.9121755902793414,
  'precision': 0.9257849084221821,
  'kappa': 0.8277169137650131,
  'auc': 0.9884083096921392}]

In [8]:
import numpy as np

accs = []
for result in results:
    accs.append(result['accuracy'])

np.mean(accs)

0.8546822428703308

In [9]:
import pickle

with open(f'results_L24SO_{model_name}.pkl', 'wb') as f:
    pickle.dump(results, f)